## [Day 18](https://adventofcode.com/2020/day18)

Alright so we have some math with 'improper' order of operations that just work left from right and do not care about the operators involved. We're just asked to evaluate these expressions and sum them.

In [40]:
import pandas as pd
import numpy as np
import itertools, re
eq = open('../inputs/d18.txt').read().splitlines()
eq[:5]

['9 + (9 * 5 + 8 * 8 + 6) * 8',
 '8 + (4 + 7) + 6 + ((5 * 4) * (5 * 7) * (3 + 7 * 4 + 5 * 7) * 4) + 4 * 8',
 '3 * 4 * (6 * 4) + 5 * 3 + 6',
 '7 * 6 + ((8 + 6 * 6 + 2 + 3) + 9 * (2 * 4 * 8 * 4)) + 8 * 6 + 9',
 '(7 + 8) * 8 + 5']

So I think my method here will be to just write a recursive function that takes in an expression and then calls the function on each parentheses within it? Seems like there are a lot of possibilities where we can get derailed on. Maybe let's start by trying to get a function to just to the parentheses-less arithmetic correct:

In [2]:
def funny_math(x):
    operators = 0
    for i, char in enumerate(x):
        if char in ['*', '+']:
            operators += 1
        # If we se a second operator, evaluate the first part and call again
        if operators == 2:
            return funny_math(str(eval(x[:i])) + x[i:])
        
    return str(eval(x))
example = '1 + 2 * 3 + 4 * 5 + 6'
funny_math(example)

'71'

Okay, that's not so bad. No we we need to make a function that can deal with the parentheses:

In [3]:
def funny_expression(x):
    # If there are no parenthese, just return:
    if x.find('(') == -1:
        return funny_math(x)    
    # I think we have to do this from left to right and count until
    # the closing parenthese are equal to the opening ones
    first_left = x.index('(')
    left, right = 1, 0
    i = 0
    while left > right:
        i += 1
        if x[first_left + i] == '(':
            left += 1
        if x[first_left+ i] == ')':
            right += 1
    reduced = x[:first_left] + funny_expression(x[first_left+1:first_left+i]) + x[first_left+i+1:]    
    return funny_expression(reduced)
        
example2 = '(4 * (5 + 6)) + (4+2)'        
funny_expression(example2)        

'50'

In [4]:
sol1 = sum([int(funny_expression(x)) for x in eq])
sol1

45840336521334

### Part 2

So for the second part we change the rules a bit so that addition is set a higher prescedence level than multiplication. I think I can just add parentheses around all the addition first and evaluate.

In [82]:
char_digits = [str(x) for x in list(range(10))]

# Let's use this function to get chunks from the position of + sign
# in either direction. pos is the index of the + sign:
def add_paren(x, pos):
    # first go forward:
    i = pos+1
    while x[i] == ' ':
        i+=1
    # first case is we hit a number:
    if x[i] in char_digits:
        while i < len(x) and x[i] in char_digits:
            i += 1
        i-= 1    
           
    # second is we hit a parentheses        
    else:
        i+=1
        left, right = 1, 0
        while left > right:
            i += 1
            if x[i] == '(':
                left += 1
            if x[i] == ')':
                right += 1
    
    # store result:
    result = x[pos:i+1] + ')' + x[i+1:]
    
    # now go left:
    i = pos-1
    while x[i] == ' ':
        i-=1
    # first case is we hit a number:
    if x[i] in char_digits:
        while i >=0 and x[i] in char_digits:
            i -= 1
        i += 1
    # second is we hit a parentheses        
    else:
        i-=1
        left, right = 0, 1
        while right > left:
            i -= 1
            if x[i] == '(':
                left += 1
            if x[i] == ')':
                right += 1
    result = x[0:i] + '(' + x[i:pos] + result 
    return result

add_paren('(3 * 4) + (34 * 2-1)', 8)    

'((3 * 4) + (34 * 2-1))'

In [83]:
add_paren('(4*2) + 7 + 9', 6)

'((4*2) + 7) + 9'

In [84]:
add_paren('432 + 7 + 9', 4)

'(432 + 7) + 9'

Now we need a way to repeatedly apply this to a string for each addition symbol:

In [87]:
def add_many_paren(x):
    # first count how many plus symbos:
    plusses = 0
    for char in x:
        if char == '+':
            plusses += 1
    # call add_paren that many times:
    for i in range(plusses):
        pos = [m.start() for m in re.finditer(r'\+', x)][i]
        x = add_paren(x, pos)
    return x
    
add_many_paren('3 + 2 * (1 + 3)')        

'(3 + 2) * ((1 + 3))'

In [96]:
sol2 = sum([eval(add_many_paren(x)) for x in eq])
sol2

328920644404583